<a href="https://colab.research.google.com/github/darshandugar2004/XrayImageClassification-TransferLearning/blob/main/Brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip -q converted_images.zip -d /content/brain_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
dataset_path = "/content/brain_dataset/converted_images"

# Define image size and batch size
img_size = (128, 128)
batch_size = 64

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,      # Increase rotation for more variation
    width_shift_range=0.2,  # Increase shift range
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,   # Add flipping for more diversity
    vertical_flip=True,     # Add vertical flip
    shear_range=0.2,        # Introduce shearing
    validation_split=0.2
)

# Load Training and Validation Data
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    subset='validation',
    shuffle=False  # Important for correct label extraction
)

# Get Class Names
class_names = list(train_data.class_indices.keys())
print("Class Names:", class_names)


Found 2452 images belonging to 3 classes.
Found 612 images belonging to 3 classes.
Class Names: ['class_0', 'class_1', 'class_2']


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint

checkpoint = ModelCheckpoint(
    filepath="/content/drive/MyDrive/brain.h5",
    save_best_only=True,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    verbose=1
)
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 1)),
    BatchNormalization(),
    Conv2D(32, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(len(class_names), activation='softmax', dtype='float32')  # Output layer
])

from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Enable XLA for faster training
tf.config.optimizer.set_jit(True)

# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train Model
history = model.fit(train_data, validation_data=val_data, epochs=25, callbacks=[lr_scheduler,checkpoint])


#model.save("brain_tumor_model.h5")
#print("Model saved successfully!")

# Evaluate Model
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy: {val_acc:.2%}")

# Get True Labels
true_labels = val_data.classes

# Get Predicted Labels
predictions = model.predict(val_data)
predicted_labels = np.argmax(predictions, axis=1)

# Print Classification Report (Precision, Recall, F1-Score)
report = classification_report(true_labels, predicted_labels, target_names=class_names)
print("\nClassification Report:\n", report)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step - accuracy: 0.4525 - loss: 1.4933
Epoch 1: val_loss improved from inf to 1.66397, saving model to /content/drive/MyDrive/brain.h5


39/39 ━━━━━━━━━━━━━━━━━━━━ 33s 501ms/step - accuracy: 0.4537 - loss: 1.4890 - val_accuracy: 0.3039 - val_loss: 1.6640 - learning_rate: 5.0000e-04
Epoch 2/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.5775 - loss: 1.0666
Epoch 2: val_loss did not improve from 1.66397
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 205ms/step - accuracy: 0.5784 - loss: 1.0648 - val_accuracy: 0.3039 - val_loss: 4.4878 - learning_rate: 5.0000e-04
Epoch 3/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.6830 - loss: 0.8432
Epoch 3: val_loss did not improve from 1.66397
39/39 ━━━━━━━━━━━━━━━━━━━━ 10s 262ms/step - accuracy: 0.6832 - loss: 0.8428 - val_accuracy: 0.3039 - val_loss: 6.5131 - learning_rate: 5.0000e-04
Epoch 4/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.7096 - loss: 0.7541
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 4: val_loss did not improve from 1.66397
39/39 ━━━━━━━━━━━━━━━━━━━━ 9s 234ms/step - accuracy: 0.7097 - loss: 0.7534 - val_accura

39/39 ━━━━━━━━━━━━━━━━━━━━ 10s 262ms/step - accuracy: 0.8360 - loss: 0.4041 - val_accuracy: 0.7288 - val_loss: 0.8913 - learning_rate: 6.2500e-05
Epoch 14/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.8297 - loss: 0.4214
Epoch 14: val_loss improved from 0.89131 to 0.63850, saving model to /content/drive/MyDrive/brain.h5


39/39 ━━━━━━━━━━━━━━━━━━━━ 9s 240ms/step - accuracy: 0.8297 - loss: 0.4211 - val_accuracy: 0.7402 - val_loss: 0.6385 - learning_rate: 6.2500e-05
Epoch 15/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.8462 - loss: 0.3824
Epoch 15: val_loss did not improve from 0.63850
39/39 ━━━━━━━━━━━━━━━━━━━━ 11s 274ms/step - accuracy: 0.8460 - loss: 0.3828 - val_accuracy: 0.7745 - val_loss: 0.6678 - learning_rate: 6.2500e-05
Epoch 16/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.8497 - loss: 0.3841
Epoch 16: val_loss did not improve from 0.63850
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 0.8496 - loss: 0.3842 - val_accuracy: 0.7288 - val_loss: 0.7717 - learning_rate: 6.2500e-05
Epoch 17/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.8444 - loss: 0.3938
Epoch 17: val_loss improved from 0.63850 to 0.41217, saving model to /content/drive/MyDrive/brain.h5


39/39 ━━━━━━━━━━━━━━━━━━━━ 10s 263ms/step - accuracy: 0.8445 - loss: 0.3939 - val_accuracy: 0.8366 - val_loss: 0.4122 - learning_rate: 6.2500e-05
Epoch 18/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.8431 - loss: 0.4239
Epoch 18: val_loss did not improve from 0.41217
39/39 ━━━━━━━━━━━━━━━━━━━━ 19s 237ms/step - accuracy: 0.8433 - loss: 0.4233 - val_accuracy: 0.7843 - val_loss: 0.5368 - learning_rate: 6.2500e-05
Epoch 19/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.8734 - loss: 0.3312
Epoch 19: val_loss improved from 0.41217 to 0.36669, saving model to /content/drive/MyDrive/brain.h5


39/39 ━━━━━━━━━━━━━━━━━━━━ 10s 243ms/step - accuracy: 0.8729 - loss: 0.3320 - val_accuracy: 0.8252 - val_loss: 0.3667 - learning_rate: 6.2500e-05
Epoch 20/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.8505 - loss: 0.3743
Epoch 20: val_loss improved from 0.36669 to 0.34092, saving model to /content/drive/MyDrive/brain.h5


39/39 ━━━━━━━━━━━━━━━━━━━━ 9s 237ms/step - accuracy: 0.8509 - loss: 0.3736 - val_accuracy: 0.8644 - val_loss: 0.3409 - learning_rate: 6.2500e-05
Epoch 21/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - accuracy: 0.8622 - loss: 0.3263
Epoch 21: val_loss improved from 0.34092 to 0.31896, saving model to /content/drive/MyDrive/brain.h5


39/39 ━━━━━━━━━━━━━━━━━━━━ 12s 299ms/step - accuracy: 0.8622 - loss: 0.3269 - val_accuracy: 0.8595 - val_loss: 0.3190 - learning_rate: 6.2500e-05
Epoch 22/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.8509 - loss: 0.3575
Epoch 22: val_loss did not improve from 0.31896
39/39 ━━━━━━━━━━━━━━━━━━━━ 9s 216ms/step - accuracy: 0.8510 - loss: 0.3575 - val_accuracy: 0.8448 - val_loss: 0.3608 - learning_rate: 6.2500e-05
Epoch 23/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.8704 - loss: 0.3212
Epoch 23: val_loss did not improve from 0.31896
39/39 ━━━━━━━━━━━━━━━━━━━━ 10s 204ms/step - accuracy: 0.8703 - loss: 0.3217 - val_accuracy: 0.7157 - val_loss: 0.7686 - learning_rate: 6.2500e-05
Epoch 24/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - accuracy: 0.8711 - loss: 0.3174
Epoch 24: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 24: val_loss did not improve from 0.31896
39/39 ━━━━━━━━━━━━━━━━━━━━ 11s 292ms/step - accuracy: 0.8711 - loss: 0.3177 - va